# Как можно оценивать RAG

## Есть разные варианты ошибок LLM:
- Лингвистические (грамматика, синтаксис и т.д.) - решаются через улучшение данных на претрейне LLM
- Этические (некорректное, предвзятое или оскорбительное поведение) - решаются через улучшение данных на претрейне LLM (***Alignment???***)
- Фактологические (связанные с неверной или устаревшей информацией) - решаются через обновление обучающего корпуса данных(дорого), использование RAG

## Как можно оценивать RAG:
1. Оценка Retrieve части
    1. **Retrieving метрики** - метрики ранжиронвания и классификации, которые оценивают выдачу контекста (MAP@k, NCDG@k и т.д.). **Но нужна разметка**.

2. Оценка генерации
    1. **Faithfulness (Answer~Context)** - Согласованность генеративного ответа к найденному контексту (можно считать в **онлайне** и **не нужна** разметка)
    2. **Relevance (Answer~Query)** - Насколько сгенерированный ответ соответствует вопросу (можно считать в **онлайне** и **не нужна** разметка)
    3. **Correctness(Answer~Answer\*)** - Измеряет степень соответствия генеративного ответа эталонному ответу (расчет только в **офлайне** и **нужна** разметка)

## Методы реализации расчета метрик:
1. **Human Eval** - разметка ассесорами (дорого, долго, но качественно), нужен **Golden Set(валидационная выборка)**
2. **LLM as a Judge** - разметка LLM (дешево, быстро, чаще хуже чем разметка ассесорами). Плохо следуют большому кол-ву инструкций [статья](https://openreview.net/pdf?id=R6q67CDBCH)
3. **Детермминированные метрики** - precision, recall, F1, BLEU, ROUGE, BertScore, слабо отражают реальность

## Фреймворки для оцеки RAG:
- **RAGAS** 

## Дополнительно:
- [Как оценивать современные RAG-системы?](https://youtu.be/fE3gDFi7tNQ?si=FbgW8avZEXK15vOI)
- [Полное руководство по оценке компонентов системы RAG: что необходимо знать](https://habr.com/ru/articles/860390/)
- [Руководство для начинающих по оценке конвейеров RAG с использованием RAGAS](https://llmarena.team/blog/rukovodstvo-dlya-nachinayushchih-po-ocenke-konvejerov-rag)
- https://arxiv.org/pdf/2405.07437

**Faithfulness (Answer~Context) Достоверность** - Согласованность генеративного ответа к найденному контексту (можно считать в **онлайне** и **не нужна** разметка) через LLM as a Judge. Необходимо сгенерить несколько ответов и отношение (кол-во ответов согласованных с контекстом) / (кол-во всех ответов), либо оценка согласованности.

**Answer Relevancy** - Насколько сгенерированный ответ соответствует вопросу (можно считать в **онлайне** и **не нужна** разметка) генерируются 3 вопроса к полученному ответу и считается косинусное расстояние между вопросами и изначальным вопросом


**Context Precision** - доля релевантных чанков контекста из всех извлеченных чанков
**Context Recall** - доля извлечённых релевантных чанков от общего числа релевантных чанков


---

# Как можно оценивать RAG

![lllm_eval.png](../imgs/lllm_eval.png)

## Мотивация
Хочется иметь понимание насколько модификации системы/модели лучше baseline решение.

P.S. Важно определить, что важнее precision(правильно отвечать на вопросы) или recall(отвечать на как можно больше вопросов)?

## Есть несколько этапов, на которых можно оценивать качество RAG
Будем рассмативать этапы с конца
- Генерация
- Ретрив (поиск чанков/документов)

![rag_schema](../imgs/rag_schema.png)

## Метрики генерации
**Можно разбить на 2 группы:**
- Метрики качества итогового ответа (BERTScore и т.д.)
- Метрики качества генерации на основе контекста (Faithfulness, Groundedness)

### Качество итогового ответа
Для этих метрик нужны ground truth (эталонные) ответы. Данные метрики показывают насколько получнный ответ схож с ground truth (эталонным) ответом.

**Можно разбить на 2 группы:**
- Non-LLM-Based метрики
    - BertScore
    - Semantic Similarity
    - BLEU (Bilingual Evaluation Understudy)
    - ROUGE (Recall-Oriented Understudy for Gisting Evaluation)
    - METEOR (Metric for Evaluation of Translation with Explicit ORdering)
- LLM-Based метрики
    - 

#### Non-LLM-Based метрики

##### BertScore
Использует контекстуальные эмбеддинги, учитывает семантическое сходство отличие от BLEU или ROUGE. BERTScore позволяет выявлять эквивалентность смысла даже при различии в словах и формулировках, учитывая синонимы и парафразы. 

Для расчета нужны эталонные ответы и ответы от RAG/LLM/системы, чьи ответы проверяем.

![bertscore](../imgs/bertscore.png)

###### Методика расчета
Метод BERTScore состоит из нескольких этапов:
1. **Получение контекстуальных эмбеддингов:** Оба текста (референсный и сгенерированный) разбиваются на токены и пропускаются через предобученную трансформерную модель (например, BERT или RoBERTa). Для каждого токена извлекается его контекстуальное векторное представление (эмбеддинг).
2. **Вычисление косинусного сходства:** Для всех пар токенов из двух текстов вычисляется косинусное сходство, и формируется матрица сходства токенов.
3. **Расчёт точности, полноты и $F_1$-меры:** На основе матрицы сходства для каждого токена в сгенерированном тексте находится наиболее похожий токен в референсном тексте, что позволяет вычислить точность (precision). Аналогично, для каждого токена референса находится самый близкий токен в сгенерированном тексте, что даёт полноту (recall). Итоговым значением BERTScore является сбалансированная $F_1$-мера, которая комбинирует точность и полноту

> $recall = \frac{1}{|x|} \sum\limits_{x_i \in x} \max\limits_{\hat{x_j} \in \hat{x}} x_i^T \hat{x_j}$

> $precision = \frac{1}{|\hat{x}|} \sum\limits_{\hat{x_j} \in \hat{x}} \max\limits_{x_i \in x} x_i^T \hat{x_j}$

> $BertScore = F_1 = 2 * \frac{precision * recal}{precision + recal}$

4. \* можно взвешивать токены по их важности (с помощью IDF-весов) и преобразовывать оценки от 0 до 1 для лучшей интерпретируемости (параметры в BertScorer)

Допустимые значения без нормировки [-1, 1]. Чем ближе BertScore к 1 тем больше полученный ответ совпадает с эталонным ответом.

###### Плюсы и минусы
**Плюсы:**
- **Учёт семантики:** Сравнивает тексты на уровне смысла, учитывая синонимы и парафразы.
- **Высокая корреляция с человеком:** Оценки BERTScore лучше согласуются с человеческими суждениями о качестве текста, чем традиционные метрики

**Минусы:**
- **Вычислительные затраты:** Расчёт на основе эмбеддингов требует значительно больше ресурсов, чем подсчёт n-грамм, и часто требует использования GPU

##### Semantic Similarity
Semantic Similarity оценивает семантическое сходство между сгенерированным ответом и эталонным ответом. В отличии от **BertScore** сравнивают эмбеддинги не слов/токенов, а ответов/предложений и уже от них берется косинусное расстояние.

Для расчета нужны эталонные ответы и ответы от RAG/LLM/системы, чьи ответы проверяем.

###### Методика расчета
Метод Semantic Similarity состоит из нескольких этапов:
1. **Получение эмбеддингов:** Оба текста (референсный и сгенерированный) пропускаются через предобученную трансформерную модель. Для текстов извлекаются векторные представления (эмбеддинги).
2. **Вычисление косинусного сходства**

Допустимые значения без нормировки [-1, 1]. Чем ближе Semantic Similarity к 1 тем больше полученный ответ совпадает с эталонным ответом.

P.S. В [RAGAS](https://github.com/vibrantlabsai/ragas/blob/main/src/ragas/metrics/collections/_semantic_similarity.py) почему-то написано, что от 0 до 1, хотя считают косинусное расстояние.

###### Плюсы и минусы
**Плюсы:**
- **Учёт семантики:** Сравнивает тексты на уровне смысла, учитывая синонимы и парафразы.
- **Высокая корреляция с человеком** 

**Минусы:**
- **Вычислительные затраты:** Расчёт на основе эмбеддингов требует значительно больше ресурсов, чем подсчёт n-грамм, и часто требует использования GPU


##### BLEU (Bilingual Evaluation Understudy)
BLEU вычисляет точность для каждого совпадающего n-грамма между полученным ответом и эталонным ответом, чтобы найти их геометрическое среднее, и применяет штраф за краткость, если это необходимо. Изначально применялся для оценки машинного перевода.

###### Методика расчета
Метод Semantic Similarity состоит из нескольких этапов:
1. **Получение эмбеддингов:** Оба текста (референсный и сгенерированный) пропускаются через предобученную трансформерную модель. Для текстов извлекаются векторные представления (эмбеддинги).
2. **Вычисление косинусного сходства**

Допустимые значения [0, 1]. Чем ближе BLEU к 1 тем больше полученный ответ совпадает с эталонным ответом.


###### Плюсы и минусы
**Плюсы:**
- **Учёт семантики:** Сравнивает тексты на уровне смысла, учитывая синонимы и парафразы.
- **Высокая корреляция с человеком** 

**Минусы:**
- **Вычислительные затраты:** Расчёт на основе эмбеддингов требует значительно больше ресурсов, чем подсчёт n-грамм, и часто требует использования GPU


##### ROUGE (Recall-Oriented Understudy for Gisting Evaluation)

##### METEOR (Metric for Evaluation of Translation with Explicit ORdering)


#### LLM-Based метрики






## Метрики ретрива
Можно разбить на 2 группы:
- Метрики качества релевантности контекста (Context Precision, Context Recall)
- Метрики оценки качества ретривера (Recall@k, Precision@k, MAP@k и т.д.)


## Фреймворки
- [BertScore](https://github.com/Tiiiger/bert_score) - для расчета bertscore
- [RAGAS](https://docs.ragas.io/en/latest/concepts/metrics/) - для метрик: 
    - 
- 

## Литература
- [[Хабр] Полное руководство по оценке компонентов системы RAG: что необходимо знать](https://habr.com/ru/articles/860390/)
- [BertScore arxiv](https://arxiv.org/pdf/1904.09675)
- [BertScore github](https://github.com/Tiiiger/bert_score)
- [RAGAS](https://docs.ragas.io/en/latest/concepts/metrics/)